In [12]:
from opik import configure 
from opik.integrations.langchain import OpikTracer 
configure(api_key="dHhF5jDOpifAMNJH9UalSjR0o",workspace="yu-li")
opik_tracer = OpikTracer(project_name="sku_match") 


OPIK: Existing Opik clients will not use updated values for "url", "api_key", "workspace".
OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/liyu/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In [13]:
# 使用langgraph接入大模型，使用大模型判断是否相似
# 输入商品名称和top3商品名称，输入相似的结果
from langchain_openai import ChatOpenAI
import os

# os.environ["LANGSMITH_API_KEY"] = "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"
# os.environ["LANGSMITH_PROJECT"] = "rag"
# os.environ["LANGSMITH_TRACING"] = "true"


qwen3_8B = ChatOpenAI(
    model="Qwen/Qwen3-8B",
    base_url="https://api.siliconflow.cn/v1",
    api_key="sk-idaudysppselrwglygkbtatkregsbxhaxypaeulbfpavrals",
)


In [14]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict


class RankSelect(BaseModel):
    """Model for match top rank. Provide json constraints."""

    match_index: int = Field(
        description="返回匹配 origin_product 商品的候选商品编号. 如果是 top1_candidate 返回 1; 如果是 top2_candidate 返回 2; 如果是 top3_candidate 返回 3；不匹配或其他情况返回0。",
    )
    match_reason: str = Field(
        description="返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。",
    )

In [15]:
rank_prompt = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的编号值。
你需要按照下面步骤进行思考处理和返回：
- 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
- 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
- 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
- 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。
- 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：
1. candidate信息为空，当前判断的候选商品不匹配。
2. 商品品牌不一致，origin_product识别品牌为可口，candidate识别为品牌为百事；商品品牌不必完全相同，语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
3. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，商品数量规则匹配需要满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位，如没有说明默认数量为1。
4. 商品名语义不符合，origin_product识别为识字卡，candidate识别为挪车卡。
5. 商品描述有冲突，origin_product识别描述冬季保暖，candidate识别描述为夏季防晒；注意，比较双方商品描述可以缺失，但是语义不能冲突，如冬天与夏天，辣味与甜味。
6. 商品信息整体语义冲突，origin_product商品信息为小型便携式冲牙器，candidate商品信息为家用取暖器暖风机；只有比较双方商品语义完全不同才判断为冲突。

请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回匹配 origin_product 商品的候选商品编号. 如果是 top1_candidate 返回 1; 如果是 top2_candidate 返回 2; 如果是 top3_candidate 返回 3；不匹配或其他情况返回0。,
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
"""

In [16]:
model = qwen3_8B.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)


In [8]:
# origin_product = "达利园 香奶味软面包  200g／袋 面包就要达利园"
# top1_candidate="达利园 香奶味法式软面包 360g/袋"
# top2_candidate="达利园 桂圆莲子八宝粥 360g/罐"
# top3_candidate="洽洽 香瓜子 160g/袋"
# top1_candidate = ""
# top2_candidate = "达利园 香奶味法式软面包 360g/袋"
# top3_candidate = ""

In [18]:
origin_product="王老吉 凉茶植物饮料 310ml_罐"
top1_candidate="【单罐】王老吉 凉茶 310ml_罐"
top2_candidate="【山姆】金晔 山楂满满山楂汁 310ml_罐"
top3_candidate="【山姆】金晔 山楂满满山楂汁 310ml_罐"

In [19]:
prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
prompt_format

'\n你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的编号值。\n你需要按照下面步骤进行思考处理和返回：\n- 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。\n- 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。\n- 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。\n- 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。\n- 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。\n\n以下是输入信息：\n<products_info>\norigin_product：王老吉 凉茶植物饮料 310ml_罐\ntop1_candidate：【单罐】王老吉 凉茶 310ml_罐\ntop2_candidate：【山姆】金晔 山楂满满山楂汁 310ml_罐\ntop3_candidate：【山姆】金晔 山楂满满山楂汁 310ml_罐\n</products_info>\n\n以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：\n1. candidate信息为空，当前判断的候选商品不匹配。\n2. 商品品牌不一致，origin_product识别品牌为可口，candidate识别为品牌为百事；商品品牌不必完全相同，语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。\n3. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为

In [20]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)

rankSelect = model.invoke([HumanMessage(content=prompt_format)],
                        config={"callbacks": [opik_tracer]})


In [21]:
rankSelect

RankSelect(match_index=1, match_reason='首先提取 origin_product 的信息：品牌为王老吉，描述为凉茶植物饮料，商品名为凉茶，规格数量为310ml_罐。然后依次提取候选商品信息。top1_candidate 的品牌为王老吉，描述为凉茶，商品名为凉茶，规格数量为310ml_罐，所有四个部分都与 origin_product 完全一致，因此匹配。top2_candidate 和 top3_candidate 的品牌为金晔，与 origin_product 的品牌不一致，因此不匹配。所以，匹配的候选商品编号是 1。')